# Numerical Experiments

With this notebook, you can experiment with some Runge-Kutta methods as described in the paper.

Run the cell below to activate the project environment containing the necessary packages.

In [ ]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

# Setup

Run this cell once to setup some basic functions.

In [ ]:
using StaticArrays
using OrdinaryDiffEq, DiffEqCallbacks
using SummationByPartsOperators
using LinearAlgebra, Printf
using Quadmath

using LaTeXStrings
import PyPlot; plt = PyPlot
using PyCall

# line cycler adapted to colourblind people
cycler = pyimport("cycler")
cycler.cycler
colours = ["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442"]
linestyles = ["-", "--", "-.", ":", "-", "--", "-."]
#markers = [">", "^", "<", "v", "+", "x", "."]
markers = ["4", "2", "3", "1", "+", "x", "."]
colourblind_cycler = (cycler.cycler(color=colours) + cycler.cycler(linestyle=linestyles))
plt.rc("axes", prop_cycle=colourblind_cycler)
plt.rc("lines", linewidth=2, markersize=12, markeredgewidth=1.5)

plt.rc("text", usetex=true)
plt.rc("text.latex", preamble="\\usepackage{newpxtext}\\usepackage{newpxmath}\\usepackage{commath}\\usepackage{mathtools}")
plt.rc("font", family="serif", size=18.)
plt.rc("savefig", dpi=200)
plt.rc("legend", fontsize="medium", fancybox=true, framealpha=0.5)

isdir("../figures") || mkdir("../figures")


# the explicit midpoint method
A = [
    0 0;
    1//2 0
]
b = [0, 1//1]
c = A * (zero(b) .+ 1)
order = 2
midpoint_rk2 = ExplicitRK(tableau=DiffEqBase.ExplicitRKTableau(A, c, b, order))

# a strongly stable second order method
A = [
    0//1 0 0 0; 
    1 0 0 0; 
    1 -1 0 0; 
    -1 1 1 0
]
b = [1//4, 1//4, 1//4, 1//4]
c = A * (zero(b) .+ 1)
order = 2
stable_rk2 = ExplicitRK(tableau=DiffEqBase.ExplicitRKTableau(A, c, b, order))

# a strongly stable third order method
A = [
    0 0 0 0 0; 
    1//2 0 0 0 0;
    1 0 0 0 0; 
    1 0 -1 0 0; 
    -3 2 1 1 0
]
b = [0, 2//3, 0, 1//6, 1//6]
c = A * (zero(b) .+ 1)
order = 3
stable_rk3 = ExplicitRK(tableau=DiffEqBase.ExplicitRKTableau(A, c, b, order))

# Time-Dependent Linear Operators

Here, the linear wave equation

\begin{equation}
\begin{aligned}
  \partial_t u(t,x) + \sin(t^2) \partial_x u(t,x) &= 0,
  && t \in (0,100), x \in [-1, 1],
  \\
  u(0,x) &= \sin(\pi x),
  && x \in [-1,1],
\end{aligned}
\end{equation}

with time-dependent coefficients and periodic boundary conditions is solved numerically.

In [ ]:
function compute_advection_sint2(alg, dt, u0func)
    T = typeof(dt)

    D = periodic_derivative_operator(1, 2, -one(T), one(T), 50)
    x = SummationByPartsOperators.grid(D)
    u0 = u0func.(x)
    f = (du,u,D,t) -> mul!(du, D, u, -sin(t^2))
    tspan = (zero(T), 100*one(T))
    ode = ODEProblem(f, u0, tspan, D)

    saveat = range(first(tspan), stop=last(tspan), length=1000)
    saving = SavingCallback((u,t,int)->integrate(u->u^2, u, D), SavedValues(T,T), saveat=saveat)
    sol = solve(ode, alg, dt=dt, callback=saving, save_everystep=false, adaptive=false)
    t = saving.affect!.saved_values.t
    energy = saving.affect!.saved_values.saveval

    t, energy, opnorm(Matrix(D))
end

In [ ]:
plt.close("all")

t, energy, opnormD = compute_advection_sint2(SSPRK33(), 1.e-5, sinpi)
@printf("The operator norm of D with respect to the standard inner product is %.3e.\n", opnormD)
plt.plot(t, energy, label="SSPRK33")

t, energy, opnormD = compute_advection_sint2(stable_rk2, 1.e-5, sinpi)
plt.plot(t, energy, label="Stable RK42")

plt.xlabel(L"t")
plt.ylabel(L"|\!| u(t) |\!|^2")
plt.legend()
plt.xlim(extrema(t))
plt.tight_layout()

plt.savefig("../figures/linear_time_dep_advection.pdf", bbox_inches="tight")

# A Special Nonlinear Autonomous System

The explicit midpoint method

\begin{align}
    y_2 & = u^n + \frac{h}{2}f(u^n), \\
    u^{n+1} & = u^n + h f(y_2)
\end{align}

conserves the energy for the ODE

\begin{align}
u_1'(t) & = \frac{-u_2}{u_1^2 + u_2^2}, \\
u_2'(t) & = \frac{ u_1}{u_1^2 + u_2^2}.
\end{align}

and all step sizes.

In [ ]:
function compute_special_problem(alg, dt)
  T = typeof(dt)

  u0 = SVector(one(T), zero(T))
  f = (u,p,t) -> SVector(-u[2], u[1]) / sum(abs2, u)
  tspan = (zero(T), 20*one(T))
  ode = ODEProblem(f, u0, tspan)
  saving = SavingCallback((u,t,int)->sum(abs2, u), SavedValues(T,T))
  sol = solve(ode, alg, dt=dt, callback=saving, save_everystep=false, adaptive=false)

  t = saving.affect!.saved_values.t
  energy = saving.affect!.saved_values.saveval

  t, energy
end


function create_plots(dt; yaxis_log=false, save_legend=false)
    algs = [SSPRK33(), midpoint_rk2, stable_rk2, stable_rk3]
    labels = ["SSPRK33", "Midpoint", "Stable RK42", "Stable RK53"]

    plt.close("all")

    for (alg, label) in zip(algs, labels)
        t, energy = compute_special_problem(alg, dt)

        plt.plot(t, energy .- first(energy), label=label)
    end

    plt.xlabel(L"t")
    plt.ylabel(L"|\!| u(t) |\!|^2 - |\!| u_0 |\!|^2")
    plt.xlim((0, 20))
    plt.gca().ticklabel_format(axis="y", scilimits=(0,0))
    plt.tight_layout()
    yaxis_log && plt.yscale("symlog", linthreshy=1.e-10)

    plt.savefig("../figures/special_problem__dt_$(-round(Int, Float64(log10(dt)))).pdf", bbox_inches="tight")

    if save_legend
        ax = plt.gca()
        plt.figure()
        handles, labels = ax.get_legend_handles_labels()
        plt.figlegend(handles, labels, loc="center", ncol=4)
        plt.savefig("../figures/special_problem__legend.pdf", bbox_inches="tight")
    end
    
    nothing
end

In [ ]:
T = Float128
create_plots(T(1.e-1), yaxis_log=false, save_legend=true)
create_plots(T(1.e-5), yaxis_log=false, save_legend=false)